In [1]:
// set up dependencies
// use local maven repository; not yet deployed to remote maven repositories.
@file:Repository("*mavenLocal")
@file:Repository("https://maven.scijava.org/content/groups/public")
@file:Repository("https://jitpack.io")

// uncomment to search in your local maven repo
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("org.ntakt:ntakt:0.1.0-SNAPSHOT")

// uncomment to search in jitpack (TODO)
// @file:DependsOn("com.github.saalfeldlab:ntakt:<tbd>")

// Add BDV vistools dependency for visualization
@file:DependsOn("sc.fiji:bigdataviewer-vistools:1.0.0-beta-21")

%use lets-plot

In [2]:
import kotlin.math.sqrt

import java.nio.file.Paths

import bdv.viewer.DisplayMode

import net.imglib2.Point
import net.imglib2.RandomAccessible as RA
import net.imglib2.RandomAccessibleInterval as RAI
import net.imglib2.type.numeric.RealType
import net.imglib2.type.numeric.integer.UnsignedByteType
import net.imglib2.type.numeric.real.DoubleType

import org.ntakt.*

In [3]:
// only do gradient in xy
val gradientDimensions = intArrayOf(0, 1)
// anisotropic data -> anisotropic cache block size
val cacheBlockSize = intArrayOf(32, 32, 3)
val threshold = 20.0

Download an HDF5 data set with 3D volumetric data, e.g. one of the [CREMI](https://cremi.org/data/) data sets. Update the paths in the next cell accordingly

In [4]:
val path = Paths.get(System.getProperty("user.home"), "Downloads", "sample_A_20160501.hdf")
val rawData = ntakt.io.n5.openHDF5<UnsignedByteType>("$path", "volumes/raw")
val rawDataExtended = rawData.extendBorder().asLongs()

In [5]:
fun <T: RealType<T>> partialDerivative(ra: RA<T>, dim: Int): RA<T> {
    val offset = Point(3).also { it.setPosition(1L, dim) }
    return (ra + offset) - (ra - offset)
}

fun <T: RealType<T>> magnitude(vararg rais: RAI<T>): RAI<DoubleType> {
    return rais
        .map { it * it }
        .reduce { d1, d2 -> d1 + d2 }
        .convert(ntakt.types.double) { s, t -> t.set(sqrt(s.realDouble)) }  
}

In [6]:
val offsets = gradientDimensions.map { d -> Point(3).also{ p -> p.setPosition(1L, d) } }
val firstPartialDerivatives = gradientDimensions.map { partialDerivative(rawDataExtended, it)[rawData].cache() }
val firstDerivativeMagnitude = magnitude(*firstPartialDerivatives.toTypedArray()).cache()
val thresholded = firstDerivativeMagnitude lt threshold
val zeroOrThreshold = thresholded.convert(rawData, UnsignedByteType()) { s1, s2, t ->
    t.setReal(if (s1.get()) s2.realDouble else 0.0)
}

In [7]:
val bdv = rawData.volatileView.show("raw").also { it.setDisplayRange(0.0, 255.0) }

// set up BDV
val vp = bdv.getBdvHandle().viewerPanel
vp.state().displayMode = DisplayMode.SINGLE

// add derivatives
for ((dim, derivative) in gradientDimensions zip firstPartialDerivatives) {
    derivative.volatileView
        .show("first-derivative-$dim", bdv=bdv)
        .setDisplayRange(0.0, 100.0)
}
firstDerivativeMagnitude.volatileView
    .show("first-derivative-magnitude", bdv=bdv)
    .setDisplayRange(0.0, 100.0)
zeroOrThreshold.cache().volatileView
    .show("thresholded", bdv=bdv)

bdv.util.BdvStackSource@66533eb0